# デモ 〜pandas vs DuckDB〜

groupbyの処理をpandasとDuckDBでそれぞれ実行

pandas

```python
pd.read_parquet("cpi.parquet").groupby("cat01_code").get_group(1).groupby(
    "area_code"
).get_group("00000").groupby("time_code", as_index=False)["value"].mean()
```    

DuckDB

```sql
SELECT time_code, avg(value)
FROM '{table}'
WHERE cat01_code = 1
AND area_code = '00000'
GROUP BY time_code
ORDER BY time_code
```

In [7]:
import subprocess
import time
from pathlib import Path

import duckdb
import memray
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

p = Path()
tracker_duckdb_path = p / "duckdb-query.bin"
tracker_pandas_path = p / "pandas-query.bin"
csv_duckdb_path = p / f"{tracker_duckdb_path.stem}.csv"
csv_pandas_path = p / f"{tracker_pandas_path.stem}.csv"

tracker_duckdb_path.unlink(missing_ok=True)
tracker_pandas_path.unlink(missing_ok=True)
with memray.Tracker(tracker_pandas_path):
    start_pandas = time.time()
    time_code_pandas = (
        pd.read_parquet("data/cpi.parquet")
        .groupby("cat01_code")
        .get_group(1)
        .groupby("area_code")
        .get_group("00000")
        .groupby("time_code", as_index=False)["value"]
        .mean()
    )
    end_pandas = time.time()
    time_diff_pandas = end_pandas - start_pandas


with memray.Tracker(tracker_duckdb_path):
    start_duckdb = time.time()
    time_code_duckdb = duckdb.sql("""
        SELECT time_code, avg(value)
        FROM 'data/cpi.parquet'
        WHERE cat01_code = 1
        AND area_code = '00000'
        GROUP BY time_code
        ORDER BY time_code
        """).to_df()
    end_duckdb = time.time()
    time_diff_duckdb = end_duckdb - start_duckdb
proc = subprocess.run(
    [
        "python",
        "-m",
        "memray",
        "transform",
        "csv",
        str(tracker_pandas_path),
        "-o",
        str(csv_pandas_path),
    ]
)
proc = subprocess.run(
    [
        "python",
        "-m",
        "memray",
        "transform",
        "csv",
        str(tracker_duckdb_path),
        "-o",
        str(csv_duckdb_path),
    ]
)
memory_pandas = pd.read_csv(csv_pandas_path).pipe(
    lambda df: df.assign(
        stack_trace_stem=df.loc[:, "stack_trace"].map(lambda x: str(x).split("/")[-1])
    )
)
memory_duckdb = pd.read_csv(csv_duckdb_path).pipe(
    lambda df: df.assign(
        stack_trace_stem=df.loc[:, "stack_trace"].map(lambda x: str(x).split("/")[-1])
    )
)
tracker_duckdb_path.unlink(missing_ok=True)
tracker_pandas_path.unlink(missing_ok=True)
csv_duckdb_path.unlink(missing_ok=True)
csv_pandas_path.unlink(missing_ok=True)

File already exists, will not overwrite: pandas-query.csv


  Calculating high watermark... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00━━━━  35% -:--:--
  Processing allocation records... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00━━━━  51% -:--:--
Wrote duckdb-query.csv


In [8]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, subplot_titles=["duckdb", "pandas"]
)
fig.add_trace(
    go.Bar(
        x=memory_duckdb.loc[:, "size"].values,
        text=memory_duckdb.loc[:, "stack_trace"].values,
        name="duckdb",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Bar(
        x=memory_pandas.loc[:, "size"].values,
        text=memory_pandas.loc[:, "stack_trace"].values,
        name="pandas",
    ),
    row=2,
    col=1,
)
fig.update_yaxes(showticklabels=False)
fig.update_layout(title="Memory", height=800)

In [9]:
px.bar(x=["pandas", "DuckDB"], y=[time_diff_pandas, time_diff_duckdb], title="実行時間")